In [13]:
from numpy import pi, linspace, diff, mean, min, max, arctan2
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame, ICRS, mean_equator_and_equinox_of_date
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [14]:
output_notebook()

Loading BokehJS ...

In [15]:
ts = load.timescale()
eph_s_name = 'de422.bsp'
t_start = ts.tt(0, 1, 1, 12, 0, 0)
eph_s = load(eph_s_name)
moon = eph_s["moon"]
earth = eph_s["earth"]
sun = eph_s["sun"]
northern_hemisphere = True

def get_colors(tt):
    return [f"#{255:02x}{int(255 * (tt[-1] - t) / (tt[-1] - tt[0])):02x}{255:02x}" for t in tt]

## Sidereal month length

In [16]:
def f_zero_lon(target, ref):
    def zero(t):
        ra, _, _  = ref.at(t).observe(target).radec()
        return  ra.radians - pi > 0
    zero.step_days = 10.0
    return zero

In [17]:
zero_lon = find_discrete(t_start, t_start + 365*100, f_zero_lon(moon, earth))
t_zero = zero_lon[0][zero_lon[1] == 0]
ra, dec, d = earth.at(t_zero).observe(moon).radec()

In [18]:
diff(t_zero)

array([np.float64(27.30761989625171), np.float64(27.378461460582912),
       np.float64(27.402844851836562), ..., np.float64(27.27176531450823),
       np.float64(27.301690638531), np.float64(27.329224816989154)],
      shape=(1335,), dtype=object)

In [19]:
colors = get_colors(t_zero.tt[:-1])
fig = figure(title="Moon sidereal month length, starting at " + t_start.tt_strftime(), x_axis_label="years", y_axis_label="Dec(deg)", width=1100)
fig.scatter((t_zero.tt[1:] - t_start.tt)/365.25, diff(t_zero.tt), fill_color=colors, line_color="red", size=7)
fig.line((t_zero.tt[1:] - t_start.tt)/365.25, diff(t_zero.tt), line_color="blue")
show(fig)

## Max Distance

In [20]:
def dist_vel_zero(target, ref):
    def zero(t):
        pos_vec, v_vec = (target.at(t) - ref.at(t)).frame_xyz_and_velocity(ecliptic_J2000_frame)
        v_x, v_y, v_z = v_vec.km_per_s
        x, y, z = pos_vec.km
        return v_x*x + v_y*y + v_z*z > 0
    zero.step_days = 10.0
    return zero

In [21]:
vel_zero = find_discrete(t_start, t_start + 365*2000, dist_vel_zero(moon, earth))
t_dist_max = vel_zero[0][vel_zero[1] == 0]
lat, lon, dist = (moon.at(t_dist_max) - earth.at(t_dist_max)).frame_latlon(ecliptic_J2000_frame)

In [22]:
fig = figure(title="Moon apogee distance", x_axis_label="years", y_axis_label="dist(km)", width=1000, height=1000)
fig.scatter((t_dist_max.tt - t_dist_max.tt[0])/365.25, dist.km, line_color="red")
show(fig)

No secular acceleration is observable over period of 2000 years